In [4]:
from datetime import datetime
#from dateutil.relativedelta import relativedelta
#import os

import json
import pandas as pd
import requests

In [5]:
# get the headers of the taxi data
url = 'https://futar.bkk.hu/api/query/v1/ws/otp/api/where/bicycle-rental.json?key=bd29005b-80cb-499b-968b-c7be1e6b3d37&version=3&appVersion=1&includeReferences=false'

response = requests.get(url)
raw_data = response.json()

with open('mydata.json', 'w') as file:
    json.dump(raw_data, file, indent=4)
    
with open('mydata2.json', 'w') as file:
    json.dump(raw_data['data'], file, indent=4)

The structure of the data Dict:

    - "currentTime": time in Unix timestamp format (milliseconds since the Unix Epoch)
    - "version": 3,
    - "status": "OK",
    - "code": 200,
    - "text": "OK",
    - "data": all the information embedded here

The "data" key contains the following subkeys:

    - "list": the detailed information about the bikes and stations
    - "outOfRange": false    ?
    - "limitExceeded": false ?
    - "class": "listWithReferences"  ?


"list" is a list of dictionaries about bikes or bike stations

An example:
```json
        {
            "id": "42711896",
            "lat": 47.51032,
            "lon": 19.028615,
            "name": "Millen\u00e1ris",
            "code": "0213",
            "type": "stele",
            "bikes": 1,
            "spaces": 999,
            "style": {
                "icon": {
                    "name": "vehicle-rental/mol-bubi"
                }
            }
        }
```

Based on the [API documentation](https://bkkfutar.docs.apiary.io/#reference/0/bicyclerental/bicyclerental):

    - id (string):    MOL Bubi állomás azonosítója, e.g. 251962
    - lat (number):   latitude, e.g. 47.511182000000005
    - lon (number):   longitude, e.g. 19.080324700000002
    - name(string):   elnevezés, e.g. 'Dózsa György út'
    - code (string):  kód, e.g. '0612' (I guess this is the unique identitier of the station/bike)
    - type (enum):    kijelző típusa, e.g. '10inch' or 'character' (these denoted as strings)
    - bikes(number):  elérhető kerékpárok száma, e.g. 5
    - spaces (number): szabad kerékpár támaszok száma. Ha 0, akkor is elhelyezhető kerékpár az állomáson a "pót" támasznál. e.g. 10

In [6]:
# create a dataframe from the detailed data
bicycle_rental = pd.DataFrame(raw_data['data']['list'])
bicycle_rental.head(10)

,id,lat,lon,name,code,type,bikes,spaces,style
0,42711896,47.510320,19.028615,Millenáris,0213,stele,1,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
1,347962837,47.509107,19.054976,860351,BIKE,NaN,1,999,{'icon': {'name': 'vehicle-rental/mol-bubi-flo...
2,42990800,47.475277,19.061092,Pázmány Péter sétány - Északi tömb,1118,stele,6,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
3,42990803,47.528740,19.069095,Kassák Lajos utca,1321,stele,5,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
4,42990804,47.525510,19.088246,Rákosrendező vasútállomás,1322,stele,2,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
5,42990801,47.469366,19.059271,Infopark - aluljáró,1119,stele,22,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
6,42990802,47.530329,19.080443,Béke tér,1320,stele,9,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
7,42990807,47.531066,19.076295,Hajdú utca - Szegedi út,1323,stele,2,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
8,42990805,47.533450,19.073750,Pap Károly utca,1324,stele,4,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}
9,42990806,47.522460,19.082262,Vágány utca-Róbert Károly körút,1402,stele,5,999,{'icon': {'name': 'vehicle-rental/mol-bubi'}}


It looks that the spaces key is not updated.

In [7]:
timestamp_milliseconds = raw_data['currentTime'] / 1000  # Convert milliseconds to seconds
formatted_date_time = datetime.utcfromtimestamp(timestamp_milliseconds).strftime('%Y-%m-%d-%H-%M')
formatted_date_time

'2024-01-19-12-53'

In [8]:
bicycle_rental.to_csv(f'bicycle_rental_data_{formatted_date_time}.csv',
                      index=False)

Filter the bike rental stations

In [15]:
bicycle_rental_stations = bicycle_rental.loc[bicycle_rental['code'] != 'BIKE']
bicycle_rental_stations.shape


(201, 9)

In [14]:
bicycle_rental.shape

(232, 9)

A simple test calculating the average number of bikes at the stations.

In [10]:
# extracteddata = []
# atl = 0
# for mylist in data["data"]["list"]:
#     if mylist["code"] != "BIKES" and mylist["spaces"] != "0":
#         extracteddata.append(mylist["bikes"])
# for i in range(len(extracteddata)):
#     atl += extracteddata[i]
# atl /= len(extracteddata)
# print(f"{str(atl)[0:4]}")
    